In [1]:
!pip install accelerate transformers==4.28.0 datasets evaluate rouge_score > /dev/null

In [2]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset, DatasetDict

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
!wget https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

--2023-06-07 06:38:56--  https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027887 (1004K) [text/plain]
Saving to: ‘1k.csv’

1k.csv              100%[===================>]   1004K  --.-KB/s    in 0.1s    

2023-06-07 06:38:57 (9.49 MB/s) - ‘1k.csv’ saved [1027887/1027887]



In [20]:
one_Kdata = pd.read_csv("/content/1k.csv")
one_Kdata.shape

(998, 2)

In [21]:
one_Kdata = one_Kdata.loc[one_Kdata.prompt.str.split(' ').str.len() > 6]

In [22]:
one_Kdata['seq1'] = one_Kdata.prompt.str.split(' ')[:4]
one_Kdata['seq2'] = one_Kdata.prompt.str.split(' ')[4:]

In [23]:
train_oneK = one_Kdata[:750].dropna()
test_oneK = one_Kdata[750:].dropna()

In [24]:
krea_train = Dataset.from_pandas(train_oneK,split='train')

In [25]:
krea_test = Dataset.from_pandas(test_oneK,split='test')

In [26]:
krea = DatasetDict()
 
krea['train'] = krea_train 
krea['test'] = krea_test

In [27]:
krea.push_to_hub("Kamaljp/krea-openprompt")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format: 0ba [00:00, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format: 0ba [00:00, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
from evaluate import load

metric = load('rouge')

In [7]:
import random
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
model_checkpoint = "t5-small"

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
max_input_length = 1024
max_target_length = 128
prefix = 'Write about '
def preprocess_function(examples):
  #print(len(examples['prompt']))
  inputs = [prefix + doc for doc in examples["prompt"]]
  model_inputs = tokenizer(inputs, 
                             max_length=max_input_length, 
                             truncation=True)
  return model_inputs

In [11]:
tokenized_krea = krea.map(preprocess_function,batched=True)

Map:   0%|          | 0/744 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

In [12]:
from transformers import (AutoModelForSeq2SeqLM, 
                          DataCollatorForSeq2Seq, 
                          Seq2SeqTrainingArguments, 
                          Seq2SeqTrainer)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [13]:
batch_size = 16

model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, 
                                       model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, 
                                           skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, 
                      labels, 
                      tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, 
                                            skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, 
                            references=decoded_labels, 
                            use_stemmer=True, 
                            use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_krea["train"],
    eval_dataset=tokenized_krea["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/Kamaljp/t5-small-finetuned-xsum into local empty directory.


In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2699 in training_step            │
│                                                                                                  │
│   2696 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2697 │   │                                                                                     │
│   2698 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2699 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2700 │   │                                                                                     │
│   2701 │   │   if self.args.n_gpu > 1:                                                           │
│   2702 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2731 in compute_loss             │
│                                                                                                  │
│   2728 │   │   │   labels = inputs.pop("labels")                                                 │
│   2729 │   │   else:                                                                             │
│   2730 │   │   │   labels = None                                                                 │
│ ❱ 2731 │   │   outputs = model(**inputs)                                                         │
│   2732 │   │   # Save past state if it exists              